In [10]:
categories = {"catether":
        {
            "id": 1,
            "name": "catether",
            "supercategory": ""
        },
        "1": {
            "id": 2,
            "name": "1",
            "supercategory": ""
        },
        "2":{
            "id": 3,
            "name": "2",
            "supercategory": ""
        },
        "3":{
            "id": 4,
            "name": "3",
            "supercategory": ""
        },
        "4":{
            "id": 5,
            "name": "4",
            "supercategory": ""
        },
        "5":{
            "id": 6,
            "name": "5",
            "supercategory": ""
        },
        "6":{
            "id": 7,
            "name": "6",
            "supercategory": ""
        },
        "7":{
            "id": 8,
            "name": "7",
            "supercategory": ""
        },
        "8":{
            "id": 9,
            "name": "8",
            "supercategory": ""
        },
        "9":{
            "id": 10,
            "name": "9",
            "supercategory": ""
        },
        "9a":{
            "id": 11,
            "name": "9a",
            "supercategory": ""
        },
        "10":{
            "id": 12,
            "name": "10",
            "supercategory": ""
        },
        "10a":{
            "id": 13,
            "name": "10a",
            "supercategory": ""
        },
        "11":{
            "id": 14,
            "name": "11",
            "supercategory": ""
        },
        "12":{
            "id": 15,
            "name": "12",
            "supercategory": ""
        },
        "12a":{
            "id": 16,
            "name": "12a",
            "supercategory": ""
        },
        "13":{
            "id": 17,
            "name": "13",
            "supercategory": ""
        },
        "14":{
            "id": 18,
            "name": "14",
            "supercategory": ""
        },
        "14a":{
            "id": 19,
            "name": "14a",
            "supercategory": ""
        },
        "15":{
            "id": 20,
            "name": "15",
            "supercategory": ""
        },
        "16":{
            "id": 21,
            "name": "16",
            "supercategory": ""
        },
        "16a":{
            "id": 22,
            "name": "16a",
            "supercategory": ""
        },
        "16b":{
            "id": 23,
            "name": "16b",
            "supercategory": ""
        },
        "16c":{
            "id": 24,
            "name": "16c",
            "supercategory": ""
        },
        "12b":{
            "id": 25,
            "name": "12b",
            "supercategory": ""
        },
        "14b":{
            "id": 26,
            "name": "14b",
            "supercategory": ""
        }
}

In [26]:
import json
import os
import xml.etree.ElementTree as ET
from collections import defaultdict

def voc_to_coco_seg(voc_path, save_path, categories=defaultdict(dict)):
    """
    Convert Pascal VOC segmentation dataset to COCO segmentation dataset format.
    
    Args:
        voc_path (str): Path to the directory containing Pascal VOC annotations.
        save_path (str): Path to save COCO formatted annotations.
    """
    annotations = []
    images = []
    category_id = 27
    annotation_id = 1
    
    # Get list of annotation files
    annotation_files = [f for f in os.listdir(voc_path) if f.endswith('.xml')]
    
    # Loop through annotation files
    for i, annotation_file in enumerate(annotation_files):

        # Parse XML file
        tree = ET.parse(os.path.join(voc_path, annotation_file))
        root = tree.getroot()
        
        # Get image dimensions
        width = int(root.find('size').find('width').text)
        height = int(root.find('size').find('height').text)
        
        # Add image information to images list
        image_id = i
        images.append({'id': image_id, 'width': width, 'height': height, 'file_name': annotation_file[:-4]+'.jpg'})
        
        # Loop through object tags
        for obj in root.findall('object'):
            
            # Get object name and bounding box coordinates
            name = obj.find('name').text
            bbox = obj.find('bndbox')
            seg = [[int(xy.text) for xy in obj.find('polygon')]]
            xmin = int(bbox.find('xmin').text)
            ymin = int(bbox.find('ymin').text)
            xmax = int(bbox.find('xmax').text)
            ymax = int(bbox.find('ymax').text)
            
            # Add category information to categories dictionary
            if name not in categories:
                categories[name] = {'id': category_id, 'name': name}
                category_id += 1
            
            # Add annotation information to annotations list
            annotation = {'id': annotation_id, 'image_id': image_id, 'category_id': categories[name]['id'],
                          'segmentation': seg, 'area': (xmax-xmin) * (ymax-ymin), 'bbox': [xmin, ymin, xmax-xmin, ymax-ymin],
                          'iscrowd': 0}
            annotations.append(annotation)
            
            annotation_id += 1
    
    # Create COCO dictionary
    coco_dict = {'images': images, 'categories': list(categories.values()), 'annotations': annotations}
    
    # Save COCO dictionary as JSON file
    with open(os.path.join(save_path, 'annotations.json'), 'w') as f:
        json.dump(coco_dict, f)


In [27]:
voc_to_coco_seg(r"voc_folder\labels", 
                r"voc_folder",
                categories)